In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import elephant
import quantities as pq
import neo
import sys
from scipy.stats import ks_2samp
plt.rcParams['figure.figsize'] = (14, 10)
import scipy.stats as stats
import itertools
import random
import sys
sys.setrecursionlimit(10000)
from pathlib import Path

In [3]:
def get_cvs(spike_data):
    """
    Get the CV for each neuron recorded. 
    
    CV = standard_deviation(ISIs)/mean(ISIs)
    """
    cvs = []
     
    spike_data = spike_data.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])

    for name, group in grouped:
            """
            Each group is senders and times for one value of senders. That is, we iterate through all 
            neurons. And the times for each neuron is in sorted order. Therefore, the cvs
            returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
            """
            t = np.asarray(group['time_ms'])
            spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
            isi = elephant.statistics.isi(spiketrain)
            cv = elephant.statistics.cv(isi)
            cvs.append(cv)
            
    return cvs

In [4]:
def get_cv_lists(exc):
    cv_list_exc = list()
    for i in range(1,11):
        exc_cvs = get_cvs(exc[i])
        cv_list_exc.append(exc_cvs)
    return cv_list_exc

In [2]:
main_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'
res = ['resolution_1_2','resolution_1_4','resolution_1_8','resolution_1_16','resolution_1_32','resolution_1_64',
      'resolution_1_128','resolution_1_256']
typ = ['brunel_rounding_1_2','brunel_rounding_equal']

In [73]:
exc = {}
for i in range(1,11):
    exc[i] = pd.read_csv(r'{}/{}/{}/large_J/brunel_rounding_True_delay_0.998046875_2.001953125_seed_{}_spikes_exc-12502-0.dat'.format(main_path,res[7],typ[1],i),
                         skiprows=2,sep='\t')

In [74]:
cv_lists = get_cv_lists(exc)

In [75]:
df = pd.DataFrame(cv_lists)

In [76]:
df.to_csv(r'{}/{}/{}/large_J/cv_exc.csv'.format(main_path,res[7],typ[1]))

For continous:

In [33]:
exc = {}
for i in range(1,11):
    exc[i] = pd.read_csv(r'{}/{}/brunel_continuous/large_J/brunel_continuous_delay_1.0_2.0_seed_{}_spikes_exc-12502-0.dat'.format(main_path,res[2],i),
                         skiprows=2,sep='\t')

In [34]:
cv_lists = get_cv_lists(exc)

In [35]:
df = pd.DataFrame(cv_lists)

In [36]:
df.to_csv(r'{}/{}/brunel_continuous/large_J/cv_exc.csv'.format(main_path,res[2]))

In [20]:
def get_frs(spike_data):
    """
    Get the FR for each neuron recorded. 
    
    FR = number of spikes fired during simulation/time of simulation
    """
    frs = []
     
    spike_data = spike_data.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])

    for name, group in grouped:
            """
            Each group is senders and times for one value of senders. That is, we iterate through all 
            neurons. And the times for each neuron is in sorted order. Therefore, the cvs
            returned must have the same order. So cvs contain cv of neuron 1, then neuron 2 .... then neuron N.
            """
            t = np.asarray(group['time_ms'])
            spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
            fr = elephant.statistics.mean_firing_rate(spiketrain)
            frs.append(fr)
            
    return frs

In [21]:
def get_fr_lists(exc):
    fr_list_exc = list()
    for i in range(1,11):
        exc_frs = get_frs(exc[i])
        fr_list_exc.append(exc_frs)
    return fr_list_exc

In [79]:
main_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data'
res = ['resolution_1_2','resolution_1_4','resolution_1_8','resolution_1_16','resolution_1_32','resolution_1_64',
      'resolution_1_128','resolution_1_256']
typ = ['brunel_rounding_1_2','brunel_rounding_equal']

In [153]:
exc = {}
for i in range(1,11):
    exc[i] = pd.read_csv(r'{}/{}/{}/large_J/brunel_rounding_True_delay_0.998046875_2.001953125_seed_{}_spikes_exc-12502-0.dat'.format(main_path,res[7],typ[1],i),
                         skiprows=2,sep='\t')

In [154]:
fr_lists = get_fr_lists(exc)

In [155]:
df = pd.DataFrame(fr_lists)

In [156]:
df.to_pickle(r'{}/{}/{}/large_J/mean_fr_exc.pkl'.format(main_path,res[7],typ[1]))

In [113]:
exc = {}
for i in range(1,11):
    exc[i] = pd.read_csv(r'{}/{}/brunel_continuous/large_J/brunel_continuous_delay_1.0_2.0_seed_{}_spikes_exc-12502-0.dat'.format(main_path,res[2],i),
                         skiprows=2,sep='\t')

In [114]:
fr_lists = get_fr_lists(exc)

In [115]:
df = pd.DataFrame(fr_lists)

In [116]:
df.to_pickle(r'{}/{}/brunel_continuous/large_J/mean_fr_exc.pkl'.format(main_path,res[2]))

In [26]:
def create_fr_dfs():
    path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data')
    for p in path.iterdir():
        # for each resolution folder
        # get list of all files of excitatory spike times in semi cont brunel in one resolution
        file = list(p.glob('brunel_continuous/*12502-0.dat'))
        if len(file)>0:
            fr_list = list()
            for f in file:
                df = pd.read_csv(f,skiprows=2,sep='\t')
                frs = get_frs(df) # get cvs of one seed value one resolution
                fr_list.append(frs)
            df = pd.DataFrame(fr_list).fillna(pq.quantity.Quantity(0,units='1/ms')) # save frs to dataframe
            df.to_pickle(r'{}\brunel_continuous\mean_fr_exc.pkl'.format(p)) # save as csv file 

In [27]:
create_fr_dfs()